In [1]:
!pip install tensorflow==2.0
!pip install transformers==2.11.0
!pip list | grep -E 'transformers|tokenizers'

     |████████████████████████████████| 86.3MB 84kB/s 
     |████████████████████████████████| 450kB 45.5MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
     |████████████████████████████████| 3.8MB 41.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=ba3cba945131d5f8735b398899785698ebf086021ca4d18bb475f03e38428475
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstal

In [ ]:
!nvidia-smi

Thu Nov 19 09:55:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
!mkdir Wordpiece_distilBert_Tokenizer

mkdir: cannot create directory ‘Wordpiece_distilBert_Tokenizer’: File exists


In [ ]:
from tokenizers import BertWordPieceTokenizer

# Initialize a tokenizer
tokenizer = BertWordPieceTokenizer()

# Then train it!
tokenizer.train(files = ["drive/MyDrive/tech_corpus_wikiAndTextBook.txt"],min_frequency=2)



In [ ]:
# And finally save it somewhere 
tokenizer.save("/content/Wordpiece_distilBert_Tokenizer")
#tokenizer.save_model('./Wordpiece_distilBert_Tokenizer')

['/content/Wordpiece_distilBert_Tokenizer/vocab.txt']

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tokenizer.encode('java is a programming language').tokens

['java', 'is', 'a', 'programming', 'language']

In [ ]:
from transformers import DistilBertTokenizerFast

trained_tokenizer = DistilBertTokenizerFast.from_pretrained("./Wordpiece_distilBert_Tokenizer", do_lower_case=True, add_special_tokens=True,
                                                       pad_to_max_length=True)

In [ ]:
from transformers import DistilBertConfig
config = DistilBertConfig()

In [ ]:
from transformers import DistilBertModel

model = DistilBertModel(config=config)

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=trained_tokenizer,
    file_path="drive/MyDrive/tech_corpus_wikiAndTextBook.txt",
    block_size=128,
)

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=trained_tokenizer)

In [ ]:
!mkdir Wordpiece_distilBert

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./Wordpiece_distilBert",
    overwrite_output_dir=True,
    num_train_epochs=1,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

# New Section

In [ ]:
model.num_parameters()

/Users/meherabhishek/PycharmProjects/AutoEvaluation/bertenv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


66362880

In [ ]:
%%time
trainer.train()

TypeError: forward() got an unexpected keyword argument 'labels'

### Sanjeev changes

In [2]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("sample_data_test.csv")
df.head()

,Unnamed: 0,correct_ans,candidate_ans,pretrained_glove_score_0.1.0,tl_glove_score_1.0.0,tl_glove_score_1.1.0,correct_ans_vec,candidate_ans_vec
0,0,Machine learning is an application of artifici...,Machine Learning is the science of getting com...,98.242176,-0.0,-0.0,"['machine', 'learning', 'is', 'an', 'applicati...","['machine', 'learning', 'is', 'the', 'science'..."
1,1,Machine learning is an application of artifici...,Machine Learning (ML) can be explained as auto...,98.644829,-0.0,-0.0,"['machine', 'learning', 'is', 'an', 'applicati...","['machine', 'learning', '(ml)', 'can', 'be', '..."
2,2,Machine learning is an application of artifici...,Data science is an inter-disciplinary field th...,97.109884,-0.0,-0.0,"['machine', 'learning', 'is', 'an', 'applicati...","['data', 'science', 'is', 'an', 'inter-discipl..."
3,3,Machine learning is an application of artifici...,software engineers are really good in coding t...,94.816715,-0.0,-0.0,"['machine', 'learning', 'is', 'an', 'applicati...","['software', 'engineers', 'are', 'really', 'go..."
4,4,Machine learning is an application of artifici...,Joint Entrance Examination – Advanced (JEE-Adv...,92.076212,-0.0,-0.0,"['machine', 'learning', 'is', 'an', 'applicati...","['joint', 'entrance', 'examination', '–', 'adv..."


In [4]:
df = df.iloc[:,1:3]
df

,correct_ans,candidate_ans
0,Machine learning is an application of artifici...,Machine Learning is the science of getting com...
1,Machine learning is an application of artifici...,Machine Learning (ML) can be explained as auto...
2,Machine learning is an application of artifici...,Data science is an inter-disciplinary field th...
3,Machine learning is an application of artifici...,software engineers are really good in coding t...
4,Machine learning is an application of artifici...,Joint Entrance Examination – Advanced (JEE-Adv...


In [22]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# Want BERT instead of distilBERT? Uncomment the following line:
# model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [24]:
tokenized = df["candidate_ans"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [25]:
tokenized.shape

(5,)

In [26]:
tokenized[5]= tokenizer.encode(text, add_special_tokens=True)

In [27]:
tokenized.shape

(6,)

In [28]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [29]:
np.array(padded).shape

(6, 183)

In [30]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(6, 183)

In [31]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [32]:
features = last_hidden_states[0][:,0,:].numpy()

In [33]:
features.shape

(6, 768)

In [23]:
text ="The president of India, officially the President of the Republic of India (IAST: Bhārat kē Rāṣhṭrapati), is the ceremonial head of state of India and the Commander-in-chief of the Indian Armed Forces.The president is indirectly elected by an electoral college comprising the Parliament of India (both houses) and the legislative assemblies of each of India's states and territories, who themselves are all directly elected."

In [36]:
print(df["candidate_ans"].values)

['Machine Learning is the science of getting computers to learn and act like humans do, and improve their learning over time in autonomous fashion, by feeding them data and information in the form of observations and real-world interactions.'
 'Machine Learning (ML) can be explained as automating and improving the learning process of computers based on their experiences without being actually programmed i.e. without any human assistance. The process starts with feeding good quality data and then training our machines(computers) by building machine learning models using the data and different algorithms. The choice of algorithms depends on what type of data do we have and what kind of task we are trying to automate.'
 'Data science is an inter-disciplinary field that uses scientific methods, processes, algorithms and systems to extract knowledge and insights from many structural and unstructured data. Data science is related to data mining, machine learning and big data.'
 'software eng

In [34]:
features.shape

(6, 768)

In [35]:
#cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(features, features))

[[1.         0.942971   0.96047986 0.81406295 0.8187834  0.7977904 ]
 [0.942971   1.0000001  0.91257393 0.8497659  0.80976015 0.75950277]
 [0.96047986 0.91257393 1.0000002  0.7982632  0.827279   0.7801221 ]
 [0.81406295 0.8497659  0.7982632  0.9999995  0.7638572  0.7189686 ]
 [0.8187834  0.80976015 0.827279   0.7638572  0.9999999  0.8490895 ]
 [0.7977904  0.75950277 0.7801221  0.7189686  0.8490895  0.99999994]]


In [21]:
#cosine similarity - bert
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(features, features))

[[1.0000001  0.8700546  0.9347966  0.6665145  0.67517495 0.725861  ]
 [0.8700546  1.0000002  0.835457   0.69221956 0.6736231  0.65473187]
 [0.9347966  0.835457   1.0000002  0.671487   0.69096136 0.7115907 ]
 [0.6665145  0.69221956 0.671487   1.0000001  0.6278155  0.6055642 ]
 [0.67517495 0.6736231  0.69096136 0.6278155  1.0000001  0.74312794]
 [0.725861   0.65473187 0.7115907  0.6055642  0.74312794 1.        ]]


In [ ]:
import pandas as pd

In [ ]:
#{‘pearson’, ‘kendall’, ‘spearman’}
df.corr(method="pearson")

In [ ]:
# jaccard similarity -> not good for semantic similarity works on count vectorizer...
# keras embedding ->glove
# 

### Keras Glove word Vector embedding

In [23]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

In [24]:
docs = [
 'Machine Learning is the science of getting computers to learn and act like humans do, and improve their learning over time in autonomous fashion, by feeding them data and information in the form of observations and real-world interactions.'
 'Machine Learning (ML) can be explained as automating and improving the learning process of computers based on their experiences without being actually programmed i.e. without any human assistance. The process starts with feeding good quality data and then training our machines(computers) by building machine learning models using the data and different algorithms. The choice of algorithms depends on what type of data do we have and what kind of task we are trying to automate.'
 'Data science is an inter-disciplinary field that uses scientific methods, processes, algorithms and systems to extract knowledge and insights from many structural and unstructured data. Data science is related to data mining, machine learning and big data.'
 'software engineers are really good in coding they are a good team player and good in mathematics'
 'Joint Entrance Examination – Advanced (JEE-Advanced), formerly the Indian Institutes of Technology-Joint Entrance Examination (IIT-JEE), is an academic examination held annually in India. It is conducted by one of the seven zonal IITs (IIT Roorkee, IIT Kharagpur, IIT Delhi, IIT Kanpur, IIT Bombay, IIT Madras, and IIT Dharwad) under the guidance of the Joint Admission Board (JAB). It is the sole prerequisite for admission to the Indian Institutes of Technology. Other universities like the Rajiv Gandhi Institute of Petroleum Technology, Indian Institute of Science Education and Research and the Indian Institute of Science also use the score obtained on the JEE-Advanced exam as the basis for admission. The examination is organised each year by one of the IITs, on a round-robin rotation pattern.',
 "The president of India, officially the President of the Republic of India (IAST: Bhārat kē Rāṣhṭrapati), is the ceremonial head of state of India and the Commander-in-chief of the Indian Armed Forces.The president is indirectly elected by an electoral college comprising the Parliament of India (both houses) and the legislative assemblies of each of India's states and territories, who themselves are all directly elected."
]

In [25]:
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])

In [26]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)

[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]


In [27]:
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
# load the whole embedding into memory
embeddings_index = dict()

[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]


In [28]:
f = open('drive/MyDrive/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 164302 word vectors.


In [30]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [31]:
embedding_matrix.shape

(15, 100)

In [2]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-11-23 12:46:22--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-11-23 12:46:22--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-11-23 12:46:23--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove

### Using SentenceBert for embdding

In [52]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [55]:
sentences = [
 'Machine Learning is the science of getting computers to learn and act like humans do, and improve their learning over time in autonomous fashion, by feeding them data and information in the form of observations and real-world interactions.',
 'Machine Learning (ML) can be explained as automating and improving the learning process of computers based on their experiences without being actually programmed i.e. without any human assistance. The process starts with feeding good quality data and then training our machines(computers) by building machine learning models using the data and different algorithms. The choice of algorithms depends on what type of data do we have and what kind of task we are trying to automate.',
 'Data science is an inter-disciplinary field that uses scientific methods, processes, algorithms and systems to extract knowledge and insights from many structural and unstructured data. Data science is related to data mining, machine learning and big data.',
 'software engineers are really good in coding they are a good team player and good in mathematics',
 'Joint Entrance Examination – Advanced (JEE-Advanced), formerly the Indian Institutes of Technology-Joint Entrance Examination (IIT-JEE), is an academic examination held annually in India. It is conducted by one of the seven zonal IITs (IIT Roorkee, IIT Kharagpur, IIT Delhi, IIT Kanpur, IIT Bombay, IIT Madras, and IIT Dharwad) under the guidance of the Joint Admission Board (JAB). It is the sole prerequisite for admission to the Indian Institutes of Technology. Other universities like the Rajiv Gandhi Institute of Petroleum Technology, Indian Institute of Science Education and Research and the Indian Institute of Science also use the score obtained on the JEE-Advanced exam as the basis for admission. The examination is organised each year by one of the IITs, on a round-robin rotation pattern.',
 "The president of India, officially the President of the Republic of India (IAST: Bhārat kē Rāṣhṭrapati), is the ceremonial head of state of India and the Commander-in-chief of the Indian Armed Forces.The president is indirectly elected by an electoral college comprising the Parliament of India (both houses) and the legislative assemblies of each of India's states and territories, who themselves are all directly elected."
]

In [56]:
len(sentences)

6

In [57]:
# Tokenization of each document
tokenized_sent = []
for s in sentences:
    tokenized_sent.append(word_tokenize(s.lower()))
tokenized_sent

[['machine',
  'learning',
  'is',
  'the',
  'science',
  'of',
  'getting',
  'computers',
  'to',
  'learn',
  'and',
  'act',
  'like',
  'humans',
  'do',
  ',',
  'and',
  'improve',
  'their',
  'learning',
  'over',
  'time',
  'in',
  'autonomous',
  'fashion',
  ',',
  'by',
  'feeding',
  'them',
  'data',
  'and',
  'information',
  'in',
  'the',
  'form',
  'of',
  'observations',
  'and',
  'real-world',
  'interactions',
  '.'],
 ['machine',
  'learning',
  '(',
  'ml',
  ')',
  'can',
  'be',
  'explained',
  'as',
  'automating',
  'and',
  'improving',
  'the',
  'learning',
  'process',
  'of',
  'computers',
  'based',
  'on',
  'their',
  'experiences',
  'without',
  'being',
  'actually',
  'programmed',
  'i.e',
  '.',
  'without',
  'any',
  'human',
  'assistance',
  '.',
  'the',
  'process',
  'starts',
  'with',
  'feeding',
  'good',
  'quality',
  'data',
  'and',
  'then',
  'training',
  'our',
  'machines',
  '(',
  'computers',
  ')',
  'by',
  'buil

In [58]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [36]:
!pip install sentence-transformers

     |████████████████████████████████| 71kB 3.3MB/s 
     |████████████████████████████████| 1.3MB 7.7MB/s 
     |████████████████████████████████| 1.1MB 43.4MB/s 
     |████████████████████████████████| 890kB 26.7MB/s 
     |████████████████████████████████| 2.9MB 48.4MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.9-cp36-none-any.whl size=101036 sha256=7c3529c08b043361c2b0b7462854b9c143f971a0f396428e3adb9d0290995377
  Stored in directory: /root/.cache/pip/wheels/fc/89/43/f2f5bc00b03ef9724b0f6254a97eaf159a4c4ddc024b33e07a
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=9e274746452798be0e9962261529cc03607ae223643f0b79b84a51f17101de80
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [59]:

from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [60]:
sentence_embeddings = sbert_model.encode(sentences)

# print('Sample BERT embedding vector - length', len(sentence_embeddings[0]))
# print('Sample BERT embedding vector - note includes negative values', sentence_embeddings[0])

In [61]:
sentence_embeddings.shape

(6, 768)

In [49]:
query = "I had pizza and pasta"
query_vec = sbert_model.encode([query])[0]

In [42]:
for sent in sentences:
  sim = cosine(query_vec, sbert_model.encode([sent])[0])
  print("Sentence = ", sent, "; similarity = ", sim)

Sentence =  I ate dinner. ; similarity =  0.71734625
Sentence =  We had a three-course meal. ; similarity =  0.6371339
Sentence =  Brad came to dinner with us. ; similarity =  0.5897908
Sentence =  He loves fish tacos. ; similarity =  0.6223935
Sentence =  In the end, we all felt like we ate too much. ; similarity =  0.41980496
Sentence =  We all agreed; it was a magnificent evening. ; similarity =  0.18081601


In [62]:
#cosine similarity - bert
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(sentence_embeddings, sentence_embeddings))

[[1.0000002  0.8131254  0.71825254 0.59160864 0.2989859  0.19182938]
 [0.8131254  0.99999976 0.58782315 0.51691    0.40263635 0.22422804]
 [0.71825254 0.58782315 0.99999994 0.4867104  0.3382322  0.24638036]
 [0.59160864 0.51691    0.4867104  1.         0.25646415 0.12867256]
 [0.2989859  0.40263635 0.3382322  0.25646415 0.99999964 0.6113453 ]
 [0.19182938 0.22422804 0.24638036 0.12867256 0.6113453  0.99999976]]
